# Para ejecutar en Google Colab en Drive

In [ ]:
# Montamos el Drive al Notebook
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Verificamos el directorio en el que nos encontramos
!pwd
!ls

In [ ]:
# Cambiamos de directorio al Drive
import os
os.chdir("drive/My Drive/PruebasCOLAB3/OF-PathPred/tests")
!ls

# Inicio de Código

In [1]:

# Imports
import sys,os,logging
''' TF_CPP_MIN_LOG_LEVEL
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printeds
3 = INFO, WARNING, and ERROR messages are not printed
'''
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
sys.path.append('../bayesian-torch')
sys.path.append('../OF-PathPred')
sys.path.append('..')

import math,numpy as np
# Important imports
import matplotlib.pyplot as plt
from path_prediction.datasets_utils import setup_loo_experiment
from path_prediction.testing_utils import evaluation_minadefde,evaluation_qualitative,evaluation_attention,plot_comparisons_minadefde, get_testing_batch
from path_prediction.training_utils import Experiment_Parameters

import torch
torch.manual_seed(1)
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms
from bayesian_torch.layers import LinearReparameterization
from bayesian_torch.layers import LSTMReparameterization

# Local models
from models.lstm_encdec_variational import lstm_encdec_variational
from utils.datasets_utils import traj_dataset


In [2]:

# In[5]:
logging.basicConfig(format='%(levelname)s: %(message)s',level=20)
# GPU
if torch.cuda.is_available():
    logging.info(torch.cuda.get_device_name(torch.cuda.current_device()))


INFO: NVIDIA GeForce GTX 1060


In [4]:
# Load the default parameters
experiment_parameters = Experiment_Parameters(add_kp=False,obstacles=False)

dataset_dir   = "../OF-PathPred/datasets/"
dataset_names = ['eth-hotel','eth-univ','ucy-zara01','ucy-zara02','ucy-univ']
#dataset_names = ['eth-hotel','eth-univ','ucy-zara01']
idTest        = 2

In [5]:
# Load the dataset and perform the split
training_data, validation_data, test_data, test_homography = setup_loo_experiment('ETH_UCY',dataset_dir,dataset_names,2,experiment_parameters,use_pickled_data=False,pickle_dir="../pickle")

INFO: Testing/validation dataset: ['ucy-zara01']
INFO: Training datasets: ['eth-hotel', 'eth-univ', 'ucy-zara02', 'ucy-univ']
INFO: Extracting data from the datasets
INFO: Sequence length (observation+prediction): 20
INFO: Reading ../OF-PathPred/datasets/ucy-zara01/mundo/mun_pos.csv
INFO: Total number of frames: 872
INFO: Total number of trajectories in this dataset: 
INFO: Add social interaction data (optical flow)
INFO: Total number of sample sequences: 
INFO: Sequence length (observation+prediction): 20
INFO: Reading ../OF-PathPred/datasets/eth-hotel/mundo/mun_pos.csv
INFO: Total number of frames: 1168
INFO: Total number of trajectories in this dataset: 
INFO: Add social interaction data (optical flow)
INFO: Reading ../OF-PathPred/datasets/eth-univ/mundo/mun_pos.csv
INFO: Total number of frames: 876
INFO: Total number of trajectories in this dataset: 
INFO: Add social interaction data (optical flow)
INFO: Reading ../OF-PathPred/datasets/ucy-zara02/mundo/mun_pos.csv
INFO: Total numbe

obs_traj:  (15481, 8, 2)
obs_traj_rel:  (15481, 8, 2)
obs_traj_theta:  (15481, 8, 1)
pred_traj:  (15481, 12, 2)
pred_traj_rel:  (15481, 12, 2)


In [6]:
num_epochs     = 2
initial_lr     = 0.03
batch_size     = 256
num_mc         = 10

In [7]:
# Creamos el dataset para torch
train_data = traj_dataset(training_data['obs_traj_rel'], training_data['pred_traj_rel'],training_data['obs_traj'], training_data['pred_traj'])
val_data = traj_dataset(validation_data['obs_traj_rel'], validation_data['pred_traj_rel'],validation_data['obs_traj'], validation_data['pred_traj'])
test_data = traj_dataset(test_data['obs_traj_rel'], test_data['pred_traj_rel'], test_data['obs_traj'], test_data['pred_traj'])

In [9]:
# Form batches
batched_train_data = torch.utils.data.DataLoader( train_data, batch_size = batch_size, shuffle=True)
batched_val_data =  torch.utils.data.DataLoader( val_data, batch_size = batch_size, shuffle=True)
batched_test_data =  torch.utils.data.DataLoader( test_data, batch_size = batch_size, shuffle=True)

In [10]:
prior_mu = 0.0
prior_sigma = 1.0
posterior_mu_init = 0.0
posterior_rho_init = -4.0
posterior_rho_init = -3.0 # 0.006715348489117967 # 0.01814992791780978 # 0.04858735157374196

len_trainset = 15481
len_valset = 1720

In [11]:
class LSTM_determinista(nn.Module):
    def __init__(self, in_size, embedding_dim, hidden_dim, output_size):
        super(LSTM_determinista, self).__init__()

        # Layers
        self.embedding = nn.Linear(in_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim)
        self.lstm2 = nn.LSTM(embedding_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, output_size)

        #self.loss_fun = nn.CrossEntropyLoss()
        self.loss_fun = nn.MSELoss()

    def forward(self, X, y, training=False):

        # Copy data
        x = X
        # Last position traj
        x_last = X[:,-1,:].view(len(x), 1, -1) 

        # Layers
        emb = self.embedding(X) # encoder for batch
        lstm_out, (hn1, cn1) = self.lstm1(emb.permute(1,0,2)) # LSTM for batch [seq_len, batch, input_size] 

        loss = 0
        pred = []
        for i, target in enumerate(y.permute(1,0,2)):
            emb_last = self.embedding(x_last) # encoder for last position
            lstm_out, (hn2, cn2) = self.lstm2(emb_last.permute(1,0,2), (hn1,cn1)) # lstm for last position with hidden states from batch

            # Decoder and Prediction
            dec = self.decoder(hn2.permute(1,0,2))
            t_pred = dec + x_last
            pred.append(t_pred)

            # Calculate of loss
            loss += self.loss_fun(t_pred, target.view(len(target), 1, -1))

            # Update the last position
            if training:
                x_last = target.view(len(target), 1, -1)
            else:
                x_last = t_pred
            hn1 = hn2
            cn1 = cn2

        # Concatenate the predictions and return
        return torch.cat(pred, dim=1), loss

    def predict(self, X, dim_pred= 1):

        # Copy data
        x = X
        # Last position traj
        x_last = X[:,-1,:].view(len(x), 1, -1) 

        # Layers
        emb = self.embedding(X) # encoder for batch
        lstm_out, (hn1, cn1) = self.lstm1(emb.permute(1,0,2)) # LSTM for batch [seq_len, batch, input_size] 

        loss = 0
        pred = []
        for i in range(dim_pred):
            emb_last = self.embedding(x_last) # encoder for last position
            lstm_out, (hn2, cn2) = self.lstm2(emb_last.permute(1,0,2), (hn1,cn1)) # lstm for last position with hidden states from batch

            # Decoder and Prediction
            dec = self.decoder(hn2.permute(1,0,2))
            t_pred = dec + x_last
            pred.append(t_pred)

            # Update the last position
            x_last = t_pred
            hn1 = hn2
            cn1 = cn2

        # Concatenate the predictions and return
        return torch.cat(pred, dim=1).detach().numpy()


In [12]:
def get_rho(sigma, delta):
    """
    sigma is represented by softplus function  'sigma = log(1 + exp(rho))' to make sure it 
    remains always positive and non-transformed 'rho' gets updated during backprop.
    """
    rho = torch.log(torch.expm1(delta * torch.abs(sigma)) + 1e-20)
    return rho

In [13]:
def MOPED_layer(layer, det_layer, delta):
    """
    Set the priors and initialize surrogate posteriors of Bayesian NN with Empirical Bayes
    MOPED (Model Priors with Empirical Bayes using Deterministic DNN)
    Reference:
    [1] Ranganath Krishnan, Mahesh Subedar, Omesh Tickoo.
        Specifying Weight Priors in Bayesian Deep Neural Networks with Empirical Bayes. AAAI 2020.
    """
    if (str(layer) == 'Conv2dReparameterization()'):
        #set the priors
        print(str(layer))
        layer.prior_weight_mu = det_layer.weight.data
        if layer.prior_bias_mu is not None:
            layer.prior_bias_mu = det_layer.bias.data

        #initialize surrogate posteriors
        layer.mu_kernel.data = det_layer.weight.data
        layer.rho_kernel.data = get_rho(det_layer.weight.data, delta)
        if layer.mu_bias is not None:
            layer.mu_bias.data = det_layer.bias.data
            layer.rho_bias.data = get_rho(det_layer.bias.data, delta)

    elif (isinstance(layer, nn.Conv2d)):
        print(str(layer))
        layer.weight.data = det_layer.weight.data
        if layer.bias is not None:
            layer.bias.data = det_layer.bias.data2

    elif (str(layer) == 'LinearReparameterization()'):
        print(str(layer))
        layer.prior_weight_mu = det_layer.weight.data
        if layer.prior_bias_mu is not None:
            layer.prior_bias_mu = det_layer.bias.data

        #initialize the surrogate posteriors

        layer.mu_weight.data = det_layer.weight.data
        layer.rho_weight.data = get_rho(det_layer.weight.data, delta)
        if layer.mu_bias is not None:
            layer.mu_bias.data = det_layer.bias.data
            layer.rho_bias.data = get_rho(det_layer.bias.data, delta)

    elif str(layer).startswith('Batch'):
        #initialize parameters
        print(str(layer))
        layer.weight.data = det_layer.weight.data
        if layer.bias is not None:
            layer.bias.data = det_layer.bias.data
        layer.running_mean.data = det_layer.running_mean.data
        layer.running_var.data = det_layer.running_var.data
        layer.num_batches_tracked.data = det_layer.num_batches_tracked.data


In [15]:
# Model Variational
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = lstm_encdec_variational(2,128,256,2,prior_mu,prior_sigma,posterior_mu_init,posterior_rho_init)
model.to(device)

lstm_encdec_variational(
  (embedding): LinearReparameterization()
  (lstm1): LSTMReparameterization(
    (ih): LinearReparameterization()
    (hh): LinearReparameterization()
  )
  (lstm2): LSTMReparameterization(
    (ih): LinearReparameterization()
    (hh): LinearReparameterization()
  )
  (decoder): LinearReparameterization()
  (loss_fun): MSELoss()
)

In [16]:
# Model Deterministic
#det_model = torch.nn.DataParallel(det_resnet.__dict__[args.arch](pretrained=True))
det_model = LSTM_determinista(2,128,256,2)

# Cargamos el Modelo
det_model.load_state_dict(torch.load("../training_checkpoints/model_deterministic5.pth"))
det_model.cuda()

FileNotFoundError: [Errno 2] No such file or directory: '../training_checkpoints/model_deterministic5.pth'

In [17]:
delta = 0.2

print("MOPED enabled")
for (idx_1, layer_1), (det_idx_1, det_layer_1) in zip( enumerate(model.children()), enumerate(det_model.children())):
    MOPED_layer(layer_1, det_layer_1, delta)
    for (idx_2, layer_2), (det_idx_2, det_layer_2) in zip( enumerate(layer_1.children()), enumerate(det_layer_1.children())):
        MOPED_layer(layer_2, det_layer_2, delta)

        for (idx_3, layer_3), (det_idx_3, det_layer_3) in zip( enumerate(layer_2.children()), enumerate(det_layer_2.children())):
            MOPED_layer(layer_3, det_layer_3, delta)
            for (idx_4, layer_4), (det_idx_4, det_layer_4) in zip( enumerate(layer_3.children()), enumerate(det_layer_3.children())):
                MOPED_layer(layer_4, det_layer_4, delta)
                for (idx_5, layer_5), (det_idx_5, det_layer_5) in zip( enumerate(layer_4.children()), enumerate(det_layer_4.children())):
                    MOPED_layer(layer_5, det_layer_5, delta)
                    for (idx_6, layer_6), (det_idx_6, det_layer_6) in zip( enumerate(layer_5.children()), enumerate(det_layer_5.children())):
                        MOPED_layer(layer_6, det_layer_6, delta)
model.state_dict()
del det_model

MOPED enabled
LinearReparameterization()
LinearReparameterization()


In [20]:
import torch.optim as optim

# Training the Model
optimizer = optim.SGD(model.parameters(), lr=initial_lr)

#optimizer = optim.SGD(model.parameters(), lr=0.015)
#optimizer = optim.SGD(model.parameters(), lr=0.03)
#optimizer = optim.SGD(model.parameters(), lr=0.05)
#optimizer = optim.SGD(model.parameters(), lr=0.08) #nan
#optimizer = optim.SGD(model.parameters(), lr=0.07) #nan 
#optimizer = optim.SGD(model.parameters(), lr=0.06) #nan 

num_mc = 10

nl_loss_ = []
kl_loss_ = []
for epoch in range(num_epochs):
    # Training
    print("----- ")
    print("epoch: ", epoch)
    error = 0
    total = 0
    M     = len(batched_train_data)
    for batch_idx, (data, target, _a , _b) in enumerate(batched_train_data):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        if torch.cuda.is_available():
            data  = data.to(device)
            target=target.to(device)

        # Step 2. Run our forward pass and compute the losses
        pred, nl_loss, kl_loss = model(data, target, training=True, num_mc=num_mc)
        
        nl_loss_.append(nl_loss.detach().item())
        kl_loss_.append(kl_loss.detach().item())
        
        # TODO: Divide by the batch size
        #pi     = (2.0**(M-batch_idx))/(2.0**M-1) #  Blundell?
        loss   = nl_loss+ kl_loss
        error += loss.detach().item()
        total += len(target)

        # Step 3. Compute the gradients, and update the parameters by
        loss.backward()
        optimizer.step()
    print("Average training loss: {:.3e}".format(error/total))

    # Validation
    error = 0
    total = 0
    M     = len(batched_val_data)
    for batch_idx, (data_val, target_val, _ , _) in enumerate(batched_val_data):
        if torch.cuda.is_available():
            data_val  = data_val.to(device)
            target_val=target_val.to(device)
        pred_val, nl_loss, kl_loss = model(data_val, target_val)
        pi     = (2.0**(M-batch_idx))/(2.0**M-1) # From Blundell
        loss   = nl_loss+ pi*kl_loss
        error += loss.detach().item()
        total += len(target_val)

    print("Average validation loss: {:.3e}".format(error/total))


----- 
epoch:  0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
plt.plot(nl_loss_,"--b", label="nl_loss")
plt.plot(kl_loss_,"--r", label="kl_loss")
plt.title("Loss training")
plt.xlabel("Iteration")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(nl_loss_,"--b", label="nl_loss")
plt.title("Loss training")
plt.xlabel("Iteration")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(kl_loss_,"--r", label="kl_loss")
plt.title("Loss training")
plt.xlabel("Iteration")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
# Guardamos el Modelo
torch.save(model.state_dict(), "../training_checkpoints/model_MOPED.pth")

In [ ]:
def plot_traj(pred_traj, obs_traj_gt, pred_traj_gt, test_homography, background):
    print("-----")
    homography = np.linalg.inv(test_homography)

    # Convert it to absolute (starting from the last observed position)
    displacement = np.cumsum(pred_traj, axis=0)
    this_pred_out_abs = displacement + np.array([obs_traj_gt[-1].numpy()])

    obs   = image_to_world_xy(obs_traj_gt, homography, flip=False)
    gt    = image_to_world_xy(pred_traj_gt, homography, flip=False)
    gt = np.concatenate([obs[-1,:].reshape((1,2)), gt],axis=0)
    tpred   = image_to_world_xy(this_pred_out_abs, homography, flip=False)
    tpred = np.concatenate([obs[-1,:].reshape((1,2)), tpred],axis=0)

    plt.figure(figsize=(12,12))
    plt.imshow(background)
    plt.plot(obs[:,0],obs[:,1],"-b", linewidth=2, label="Observations")
    plt.plot(gt[:,0], gt[:,1],"-r", linewidth=2, label="Ground truth")
    plt.plot(tpred[:,0],tpred[:,1],"-g", linewidth=2, label="Prediction")
    plt.legend()
    plt.title('Trajectory samples')
    plt.show()


In [ ]:
from obstacles import image_to_world_xy

num_samples = 30
num_monte_carlo = 20
i = 1 # sample of batch
bck = np.load('background.npy')

# Testing
cont = 0
for batch_idx, (datarel_test, targetrel_test, data_test, target_test) in enumerate(batched_test_data):
    print("-----")
    print(cont)
    if torch.cuda.is_available():
        datarel_test  = datarel_test.to(device)
        targetrel_test= targetrel_test.to(device)
        data_test     = data_test.to(device)
        target_test   = target_test.to(device)        
    homography = np.linalg.inv(test_homography)
    
    obs_traj_gt  = data_test[i,:,:]
    pred_traj_gt = target_test[i,:,:]
    obs   = image_to_world_xy(obs_traj_gt.cpu(), homography, flip=False)
    gt    = image_to_world_xy(pred_traj_gt.cpu(), homography, flip=False)
    gt = np.concatenate([obs[-1,:].reshape((1,2)), gt], axis=0)

    plt.figure(figsize=(12,12))
    plt.imshow(bck)
    plt.plot(obs[:,0],obs[:,1],"-b", linewidth=2, label="Observations")
    plt.plot(gt[:,0], gt[:,1],"-r", linewidth=2, label="Ground truth")

    # prediction
    for mc_run in range(num_monte_carlo):
        pred, kl = model.predict(datarel_test, dim_pred=12)
        # ploting 
        #plot_traj(pred[i,:,:], data_test[i,:,:], target_test[i,:,:], test_homography, bck)

        pred_traj = pred[i,:,:]

        # Convert it to absolute (starting from the last observed position)
        displacement      = np.cumsum(pred_traj, axis=0)
        this_pred_out_abs = displacement + np.array([obs_traj_gt[-1].cpu().numpy()])

        tpred = image_to_world_xy(this_pred_out_abs, homography, flip=False)
        tpred = np.concatenate([obs[-1,:].reshape((1,2)), tpred], axis=0)

        if mc_run == 0:
            plt.plot(tpred[:,0],tpred[:,1],"-g", linewidth=2, label="Prediction")
        else:
            plt.plot(tpred[:,0],tpred[:,1],"-g", linewidth=2)

    plt.legend()
    plt.title('Trajectory samples')
    plt.savefig("traj_variational_1_4"+str(cont)+".pdf")
    plt.show()

    cont += 1
        
    if cont == num_samples:
        break